In [1]:
import pandas as pd 
def generate_features(df):
    features = ['seconds_in_bucket', 'imbalance_buy_sell_flag',
               'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
                'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2'
               ]
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                df[f'{a}_{b}_imb'] = df.eval(f'({a}-{b})/({a}+{b})')
                features.append(f'{a}_{b}_imb')    
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = df[[a,b,c]].max(axis=1)
                    min_ = df[[a,b,c]].min(axis=1)
                    mid_ = df[[a,b,c]].sum(axis=1)-min_-max_

                    df[f'{a}_{b}_{c}_imb2'] = (max_-mid_)/(mid_-min_)
                    features.append(f'{a}_{b}_{c}_imb2')
    df = df.replace(float('inf'),10.0**8)
    df = df.replace(-float('inf'),-10.0**8)
    df = df.fillna(0)
    return df[features]

TRAINING = False
if TRAINING:
    df_train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
    df_ = generate_features(df_train)

In [2]:
LGBparams = {#"device" : "gpu",
                        'objective'         : 'regression_l1',
                        'boosting_type'     : 'gbdt',
                        'random_state'      : 42,
                        'colsample_bytree'  : 0.7,
                        'subsample'         : 0.65,
                        'learning_rate'     : 0.065,
                        'max_depth'         : 6,
                        'n_estimators'      : 500,
                        'num_leaves'        : 150,  
                        'reg_alpha'         : 0.01,
                        'reg_lambda'        : 3.25,
                        'verbose'           : -1,}
XGBparams = {#'tree_method':"hist", 'device':"cuda",
                      'objective'          : 'reg:absoluteerror',
                      'random_state'       : 42,
                      'colsample_bytree'   : 0.7,
                      'learning_rate'      : 0.07,
                      'max_depth'          : 6,
                      'n_estimators'       : 500,                         
                      'reg_alpha'          : 0.025,
                      'reg_lambda'         : 1.75,
                      'min_child_weight'   : 1000,
                      'early_stopping_rounds' : 100}
CGBparams = {#'task_type': 'GPU',
                   'objective'           : "MAE",
                   'eval_metric'         : "MAE",
                   'bagging_temperature' : 0.5,
                   'colsample_bylevel'   : 0.7,
                   'iterations'          :500,
                   'learning_rate'       : 0.065,
                   'od_wait'             : 25,
                   'max_depth'           : 7,
                   'l2_leaf_reg'         : 1.5,
                   'min_data_in_leaf'    : 1000,
                   'random_strength'     : 0.65, 
                   'use_best_model'      : True,
                  }

In [3]:
import lightgbm as lgb 
import xgboost as xgb 
import catboost as cbt 
import numpy as np 
import joblib 
import os 

os.system('mkdir models')

model_path ='/kaggle/input/trainedmodels'

N_fold = 6

if TRAINING:
    X = df_.values
    Y = df_train['target'].values

    X = X[np.isfinite(Y)]
    Y = Y[np.isfinite(Y)]

    index = np.arange(len(X))

models = []

def train(model_dict, modelname='lgb'):
    if TRAINING:
        model = model_dict[modelname]
        model.fit(X[index%N_fold!=i], Y[index%N_fold!=i], 
                    eval_set=[(X[index%N_fold==i], Y[index%N_fold==i])], 
                    verbose=10, 
                    )
        models.append(model)
        joblib.dump(model, f'./models/{modelname}_{i}.model')
    else:
        if modelname == 'lgb':
            k = i * 3
        elif modelname == 'xgb':
            k = i * 3 + 1
        else:
            k = i * 3 + 2
        models.append(joblib.load(f'{model_path}/{k}.model'))
    return 

model_dict = {
    'lgb': lgb.LGBMRegressor(**LGBparams),
    'xgb': xgb.XGBRegressor(**XGBparams),
    'cbt': cbt.CatBoostRegressor(**CGBparams),

}

for i in range(N_fold):
    train(model_dict, 'lgb')
    train(model_dict, 'xgb')
    train(model_dict, 'cbt')


In [4]:
"""i = 0
os.system('mkdir models2')
for model in models:
    joblib.dump(model, f'./models2/{i}.model')
    i += 1
"""

"i = 0\nos.system('mkdir models2')\nfor model in models:\n    joblib.dump(model, f'./models2/{i}.model')\n    i += 1\n"

In [5]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [6]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    feat = generate_features(test)
    
    sample_prediction['target'] = np.mean([model.predict(feat) for model in models],0)
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
